In [ ]:
import sys
dir1 = '/Users/erihe/OneDrive - NTNU/'
if not dir1 in sys.path: sys.path.append(dir1)
from utils_new import *

In [ ]:

data_dir = '/Users/erihe/OneDrive - NTNU/UMAPH/Data'
session = 'Mouse28-140313'
data_path = data_dir + '/' + session + '/'
f = np.load(data_path + 'data.npz', allow_pickle = True)
data = f['data'][()]
f.close()        
angle_list_orig = np.array(read_numerical_file(data_dir + '/' + session + '/' + session + '.ang', 'float', 'multiple'))
angle_list = np.array(angle_list_orig[:,1])
angle_list[angle_list < -0.5] = np.nan
angle_times  =angle_list_orig[:,0]
data['angle_list'] = angle_list
data['angle_times'] = angle_times
spikes =data['spike_times']
samp_rate = data['pos_sampling_rate']

it = 0

hd = data['angle_list']
pos_list_orig = np.array(read_numerical_file(data_dir + '/'+ session + '/' + session + '.pos', 'float', 'multiple'))
t = pos_list_orig[:,0]
x = pos_list_orig[:,1]
y = pos_list_orig[:,2]

dt_curr = 0.1
res = 10000
min_time, max_time = data['state_times']['Wake'][0]
tt, xx, yy, speed, aa =  get_pos(x, y, t, hd, 
                                 dt_orig = dt_curr, res = res, min_time = min_time, max_time = max_time)


In [ ]:
sspikes_all = {}
spk_all = {}
for brain_state in ['Wake', 'REM', 'SWS']:
    print(brain_state)
    t_curr =  data['state_times'][brain_state]
    max_times = {}
    spk_bin = np.zeros((0,len(spikes)))
    sspk = np.zeros((0,len(spikes)))
    if brain_state == 'SWS':
        sigma = 0.5
    else:
        sigma = 1
    for tminmax in t_curr:
        min_time = tminmax[0]
        max_time = tminmax[1]
        spk_count1, tt = spk_count(spikes, min_time = min_time, max_time = max_time, dt_orig = dt_curr, res = res)
        sspk1, __ = firing_rate(spikes, sigma = sigma, min_time = min_time, max_time = max_time, 
                                 dt_orig = dt_curr, res = res)
        spk_bin = np.concatenate((spk_bin, spk_count1),0)
        sspk = np.concatenate((sspk, sspk1),0)
    sspikes_all[brain_state] = sspk
    spk_all[brain_state] = spk_bin

In [ ]:
num_neurons = len(spikes)
Xcorr = {}
for brain_state in ['Wake', 'REM', 'SWS']:
    sspikes = np.sqrt(sspikes_all[brain_state])
    t0 = time.time()
    Xcorr[brain_state] =  cross_corr_dist(sspikes, lencorr = 30)

sspikes = np.sqrt(sspikes_all['Wake'])
rmap = np.zeros((num_neurons, 25,25))
acorr = np.zeros((num_neurons, 25,25))
for i in range(num_neurons):
    currmap = binned_statistic_2d(xx[:len(sspikes)],yy[:len(sspikes)], 
                                  sspikes[:,i], statistic = 'mean', bins = 25)[0]
    nans = np.isnan(currmap)
    currmap[nans] = np.mean(currmap[~nans])
    currmap = gaussian_filter(currmap, 1.)
    acorr[i,:,:] = pearson_correlate2d(currmap, currmap)
    rmap[i,:,:] = currmap



In [ ]:
    
t_acorrs = {}
spk_times = {}
for brain_state in ['Wake', 'REM', 'SWS']:
    t_curr =  data['state_times'][brain_state]
    spk_times[brain_state] = []
    for i in range(num_neurons):
        spk_times[brain_state].append([])
    for tminmax in t_curr:
        print(tminmax)
        for i, s in enumerate(spikes):
            spk_times[brain_state][i].extend(spikes[s][(spikes[s]>=tminmax[0]) & (spikes[s]<tminmax[1])])
            
for brain_state in ['Wake', 'REM', 'SWS']:
    print(brain_state)
    t_curr =  data['state_times'][brain_state]
    t_acorr = get_temporal_acorr(spk_times[brain_state], maxspikes = 1000, inds = range(num_neurons))
    t_acorr = t_acorr[:, 200:]
    t_acorr = t_acorr.astype(float)
    for i in range(len(t_acorr[:,0])):
        t_acorr[i,:] = t_acorr[i,:]/t_acorr[i,0]
    t_acorr[:,0] = 0
    t_acorr = gaussian_filter1d(t_acorr[:, :],sigma = 2, axis = 1)
    t_acorrs[brain_state] = t_acorr.copy()




In [ ]:
d1 = np.zeros((num_neurons,num_neurons))
for x in Xcorr:    
    xtmp = Xcorr[x]
    dd1 = squareform(pdist(np.square(xtmp), 'correlation'))
    d1 += dd1
infs = np.isinf(d1)
d1[infs] = np.max(d1[~infs])*2
d1 /= len(Xcorr)
dvals = d1[np.triu_indices(len(d1),1)]


In [ ]:
thr = 0.6
ind1 = get_ind(d1,thr, linkage = 'average', bPlot = True)
numinds = np.bincount(ind1)
rel_inds = np.where((numinds>=20) & (numinds<=150))[0] 


In [ ]:
hd_info = np.zeros(num_neurons)
for i in range(num_neurons):
    spktmp = sspikes_all['Wake'][:,i].copy()
    mtot, __,  circ  = binned_statistic(aa, 
                                        spktmp,
                                        statistic = 'mean', 
                                        bins = 30)
    mu = np.mean(spktmp)
    hd_info[i] = information_score_1d(mtot, circ-1, mu)
num_neurons = len(spk_times['Wake'])
meanRate_of = np.zeros((num_neurons))
for i in range(num_neurons):
    spk = spk_times['Wake'][i].copy()
    meanRate_of[i] = len(spk)
min_of_1, max_of_1 =  data['state_times'][brain_state][0]
meanRate_of/=(max_of_1-min_of_1)
scores = (('rmap', rmap),
          ('acorr2d', acorr),
          ('tacorrs', t_acorr),
         ('hd_info', hd_info),
        ('sum',meanRate_of),
    )


In [ ]:
rel_inds = np.where((numinds>=15) & (numinds<=150))[0] 
for ii in rel_inds:
    print('Cluster ' + str(ii),' num neurons ' + str(sum(ind1==ii)))
    ################ Plot cluster stats ##############
    #mod_ind1s = np.where(isall['is_conj_S1']==ii)[0]
    mod_ind1s = np.where(ind1==ii)[0]
    scores_cluster(sspikes_all['Wake'][:, mod_ind1s][np.arange(0, len(xx), 10)], scores, 
                   xx = xx[np.arange(0, len(xx), 10)], yy = yy[np.arange(0, len(xx), 10)], 
                   inds = mod_ind1s, num_example = 5, 
                   dim = min(len(mod_ind1s),10))
    plt.show()
    print('')


In [ ]:
pthr = 50
k = 1000
maxdim = 1
n_all = [700, ]
dim_all = [3, ]
eps_all = [0.1, ]
hdmetric = np.zeros((len(rel_inds), 3, len(dim_all), len(eps_all), len(n_all)))
for indit, ii in enumerate(rel_inds):
    for fiit, brain_state in enumerate(['SWS', ]):            
        fiit += 1
        print(brain_state)
        mod_ind1s = np.where(ind1==ii)[0]
        spk1 = preprocessing.scale(sspikes_all[brain_state][:,mod_ind1s],axis = 0)
        movetimes0 = np.arange(0,len(spk1), 3)#int((len(spk1)/50000)))
        sumspk = np.sum(spk1[movetimes0],1)
        movetimes0 = movetimes0[np.sort(np.argsort(sumspk)[-20000:])]
        print(len(movetimes0))
        spk1 = spk1[movetimes0]

        for dimit, dim in enumerate(dim_all):
            dim_red_spikes_move_scaled_bu, e1, e2, var_exp = pca(spk1, dim = dim)
            dim_red_spikes_move_scaled_bu /= np.sqrt(e2[:dim])                        
            startindex = np.argmax(np.sum(np.abs(dim_red_spikes_move_scaled_bu),1))
            for epsit, eps in enumerate(eps_all):
                movetimes1 = radial_downsampling(dim_red_spikes_move_scaled_bu, epsilon = eps, 
                    startindex = startindex)
                print(len(movetimes1))
                indstemp = []
                for nit, n_points in enumerate(n_all):
                    if len(movetimes1)<n_points:
                        continue
                    if nit == 0:                        
                        indstemp  = sample_denoising(dim_red_spikes_move_scaled_bu[movetimes1,:],  k, 
                                                           n_points, 1, metric = 'cosine')[0]
                        indstemp = movetimes1[indstemp]
                        dim_red_spikes_move_scaled = dim_red_spikes_move_scaled_bu[indstemp,:]
                    elif (nit > 0) & (len(indstemp)==0):
                        continue
                    else:
                        dim_red_spikes_move_scaled = dim_red_spikes_move_scaled[:n_points]
                        indstemp = indstemp[:n_points]
                    X = squareform(pdist(dim_red_spikes_move_scaled[:,:], 'cosine'))
                    thresh = np.max(X[~np.isinf(X)])    
                    persistence = ripser(X, maxdim=1, coeff=47, do_cocycles= True, distance_matrix = True, thresh = thresh)   
                    try: 
                        coords_ds, coords_ds_consistent = get_coords_consistent(persistence, coeff = 47, ph_classes = [0,1], bConsistent = True)
                    except:
                        continue
                    dgms = persistence['dgms']
                    plt.figure()
                    plot_diagrams(dgms, list(np.arange(maxdim+1)), lifetime = True)
                    plt.show()
                    plot_barcode(dgms)
                    fig, ax = plt.subplots(1,1)
                    ax.plot(var_exp[:15])
                    ax.set_aspect(1/ax.get_data_ratio())
                    plt.show()

                    fig, ax = plt.subplots(1,4, figsize = (10,5), dpi = 120)
                    for i in range(len(coords_ds)):
                        ax[i].plot(coords_ds[i,np.argsort(coords_ds[i,:])])
                    ax[3].scatter(*coords_ds[:2,:], s = 100)
                    for i in range(4):
                        ax[i].set_aspect(1/ax[i].get_data_ratio())

                    coords_mod1 = get_coords_all(spk1, 
                                                 coords_ds,
                                                 np.arange(len(spk1)),                             
                                                 indstemp,
                                                 dim = dim, 
                                                spk2 = preprocessing.scale(sspikes_all['Wake'][:, mod_ind1s], 
                                                                           axis = 0),
                                                 bPCA = True,
                                                 bPred = False)
                    coords_mod1 = coords_mod1%(2*np.pi)
                    fig1, axs1 = plt.subplots(1,2, figsize = (6,4), dpi = 120)
                    plt.show()
                    for c in [0,]:
                        coords_mod1_tmp = 2*np.pi - coords_mod1

                        dt0 = np.arctan2(np.mean(np.sin(coords_mod1[:,c]-aa)),
                                         np.mean(np.cos(coords_mod1[:,c]-aa)))
                        dt1 = np.arctan2(np.mean(np.sin(coords_mod1_tmp[:,c]-aa)),
                                         np.mean(np.cos(coords_mod1_tmp[:,c]-aa)))
                        
                        coords_mod1[:,c] = (coords_mod1[:,c] - dt0)%(2*np.pi)
                        coords_mod1_tmp[:,c] = (coords_mod1_tmp[:,c] - dt1)%(2*np.pi)
                        
                        ds0 = np.mean(np.abs(np.arctan2(np.sin(coords_mod1[:,c]-aa),
                                         np.cos(coords_mod1[:,c]-aa))))
                        ds1 = np.mean(np.abs(np.arctan2(np.sin(coords_mod1_tmp[:,c]-aa),
                                         np.cos(coords_mod1_tmp[:,c]-aa))))
                        if ds1 < ds0:
                            coords_mod1[:,c] = coords_mod1_tmp[:,c]
                            hdmetric[indit, fiit, dimit, epsit, nit] = ds1
                            print('mean hd', ds1)
                        else:
                            hdmetric[indit, fiit, dimit, epsit, nit] = ds0
                            print('mean hd', ds0)
                        plt.figure()
                        plt.plot(aa[1000:3000])
                        plt.plot(coords_mod1[1000:3000,c])
                    plt.show()

In [ ]:
for indit, ii in enumerate(rel_inds):
    for fiit, brain_state in enumerate(['SWS', ]):            
        fiit += 1
        print(brain_state)
        mod_ind1s = np.where(ind1==ii)[0]
        spk1 = preprocessing.scale(sspikes_all[brain_state][:,mod_ind1s],axis = 0)
        movetimes0 = np.arange(0,len(spk1), 3)#int((len(spk1)/50000)))
        sumspk = np.sum(spk1[movetimes0],1)
        movetimes0 = movetimes0[np.sort(np.argsort(sumspk)[-20000:])]
        print(len(movetimes0))
        spk1 = spk1[movetimes0]


In [ ]:
fig, ax = plt.subplots(1,1, dpi = 120, figsize = (15,3))
ax.plot(aa[2000:3500], lw = 3, c = [0.2,0.2,0.2])
it = 2
ax.plot(coords_mod1[2000:3500,c], lw = 3, c = [0.,0.4*it,0.4*it])
ax.spines[['bottom', 'left']].set_linewidth(4)
ax.tick_params(width=4, length=10)
ax.set_xticks([0, 750,1500], [])
ax.set_yticks([0, np.pi,2*np.pi], [])
#ax.set_aspect(1/ax.get_data_ratio())

ax.spines[['right', 'top']].set_visible(False)

In [ ]:
spk1 = preprocessing.scale(sspikes_all[brain_state][:,mod_ind1s],axis = 0)
coords_mod2 = get_coords_all(spk1, 
                             coords_ds,
                             movetimes0,                             
                             indstemp,
                             dim = dim, 
                             bPCA = True,
                             bPred = False)
coords_mod2[:,c] = ((2*np.pi-coords_mod2[:,c])-dt1)%(2*np.pi)


In [ ]:
spk1 = preprocessing.scale(sspikes_all[brain_state][:,mod_ind1s],axis = 0)
coords_mod1 = get_coords_all(spk1, 
                             coords_ds,
                             movetimes0,                             
                             indstemp,
                             spk2 = preprocessing.scale(sspikes_all['Wake'][:, mod_ind1s], 
                                                       axis = 0),
                             dim = dim, 
                             bPCA = True,
                             bPred = False)
coords_mod1[:,c] = ((2*np.pi-coords_mod1[:,c])-dt1)%(2*np.pi)


In [ ]:
spk1 = sspikes_all['Wake'][:,mod_ind1s]
spk2 = sspikes_all['SWS'][:,mod_ind1s]


In [ ]:
from matplotlib import gridspec
numfigs = 1
numw = 12
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs)*1.05), np.ceil(numh*1.1)), dpi = 200)
nw = 0
numbins1 = 30
num_neurons = len(spk1[0,:])
for nn, n in enumerate(range(num_neurons)):
    nnn = nn%numh
    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = 0.1)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.1,wspace = 0.1)
        nw += 1
    xnum = 0

    ax = plt.subplot(gs2[nnn,xnum], projection = 'polar')
    xnum += 1
    
    bins = np.linspace(0,2*np.pi,31)
    
    
    rec = binned_statistic(aa, spk1[:,n], 
                           statistic = 'mean', bins=bins, range=None)[0]
    rec[np.isnan(rec)] = 0
    rec /= np.max(rec)
    rec = gaussian_filter1d(rec, mode = 'wrap', sigma = 1)
    ax.plot(bins, np.concatenate(([rec[-1]], rec)), 
            lw = 1.5, ls = '-', alpha = 0.9, c = [0.2,0.2,0.2], label = 'Rec Wake')

    it = 2
    dec = binned_statistic(coords_mod2[:,0], spk2[:,n], statistic = 'mean', bins=bins, range=None)[0]
    dec[np.isnan(dec)] = 0
    dec /= np.max(dec)
    dec = gaussian_filter1d(dec, mode = 'wrap', sigma = 1)
    ax.plot(bins, np.concatenate(([dec[-1]], dec)), 
            lw = 1.5, ls = '-', alpha = 0.9, c = [0.,0.2*it,0.2*it], label = 'Dec OF')

    ax.set_rticks([0.33, 0.66,1]) 
    ax.set_yticklabels('')
    ax.set_xticklabels('')
    ax.set_rlim([0,1.1])

In [ ]:
from matplotlib import gridspec
hd_cells = [15, 35, 24, 17]
numfigs = 1
numw = len(hd_cells)
num_neurons = numw
numh = int(np.ceil(num_neurons/numw))
outer1 = gridspec.GridSpec(1, numw)
fig = plt.figure(figsize=(np.ceil((numw*numfigs)*1.05), np.ceil(numh*1.1)), dpi = 200)
nw = 0
numbins1 = 30
for nn, n in enumerate(hd_cells):
    nnn = nn%numh
    if nnn == 0:
        outer2 = gridspec.GridSpecFromSubplotSpec(1, 1, subplot_spec = outer1[nw], wspace = 0.1)
        gs2 = gridspec.GridSpecFromSubplotSpec(numh, numfigs, subplot_spec = outer2[0], hspace = 0.1,wspace = 0.1)
        nw += 1
    xnum = 0

    ax = plt.subplot(gs2[nnn,xnum], projection = 'polar')
    xnum += 1
    
    bins = np.linspace(0,2*np.pi,31)
    
    
    rec = binned_statistic(aa, spk1[:,n], 
                           statistic = 'mean', bins=bins, range=None)[0]
    rec[np.isnan(rec)] = 0
    rec /= np.max(rec)
    rec = gaussian_filter1d(rec, mode = 'wrap', sigma = 1)
    ax.plot(bins, np.concatenate(([rec[-1]], rec)), 
            lw = 2, ls = '-', alpha = 0.9, c = [0.2,0.2,0.2], label = 'Rec Wake')

    it = 2
    dec = binned_statistic(coords_mod2[:,0], spk2[:,n], statistic = 'mean', bins=bins, range=None)[0]
    dec[np.isnan(dec)] = 0
    dec /= np.max(dec)
    dec = gaussian_filter1d(dec, mode = 'wrap', sigma = 1)
    ax.plot(bins, np.concatenate(([dec[-1]], dec)), 
            lw = 2, ls = '-', alpha = 0.9, c = [0.,0.4*it,0.4*it], label = 'Dec OF')

    ax.set_rticks([0.33, 0.66,1]) 
    ax.set_yticklabels('')
    ax.set_xticklabels('')
    ax.set_rlim([0,1.1])
#    ax.set_title(str(nn))